In [1]:
import pandas as pd
import numpy as np

In [7]:
#read in cleaned combined data
NewDatasetCombine = pd.read_csv('../../../Data/cleanedData/CombinedCleanedDatasets.csv', low_memory=False)

#get dataframe of unique guide/gene combinations across datasets
UniqueNewDatasetCombine = NewDatasetCombine[['Target Gene Symbol',
                                             'chromosome',
                                             'sgRNA Context Sequence',
                                             'sgRNA \'Cut\' Position']].drop_duplicates()

#alter chromosome notation to be compatible with DHS files
UniqueNewDatasetCombine['chromosome'] = UniqueNewDatasetCombine['chromosome'].replace('chr24','chrY')
UniqueNewDatasetCombine['chromosome'] = UniqueNewDatasetCombine['chromosome'].replace('chr23','chrX')
UniqueNewDatasetCombine

,Target Gene Symbol,chromosome,sgRNA Context Sequence,sgRNA 'Cut' Position
0,LARS2,chr3,CAGAAAAAAAAAAACAGGACACAGGGGAGA,45389244.0
3,COQ2,chr4,ATAAAAAAAAAAAAGGGCACCAAGTGGCCA,83285687.0
6,IGBP1,chrX,AATAAAAAAAAAAAGTGATCAGTTTGGAAA,70133000.0
9,YEATS4,chr12,AAAAAAAAAAAAAATTAACGTGCCAGGGGC,69360661.0
12,COPS3,chr17,AAAAAAAAAAAAAGCCGTGTGAGCTGGCGG,17280422.0
...,...,...,...,...
257411,WDR11,chr10,GGCTAGGACCTCTACTACAATGAATGGGAG,120855890.0
257412,WDR11,chr10,GCTAGGACCTCTACTACAATGAATGGGAGA,120855891.0
257413,WDR11,chr10,CAGTCTTTCATCAAGTCTGATGTAAGGTAT,120855971.0
257414,WDR11,chr10,TTTATGTAGGTCCAAGTTTCAGTCTGGTAT,120856094.0


In [8]:
#divide unique guide/gene combinations into a separate file for each gene
for grp, each_csv in UniqueNewDatasetCombine.groupby(['Target Gene Symbol','chromosome']):
    each_csv[['sgRNA Context Sequence', 'sgRNA \'Cut\' Position']].to_csv(f"../../../Data/DHS/perGeneGuidePositions/{grp[1]}_{grp[0]}.csv",
                                                                          index = False)

### Download DHS files and manually combine replicates

Download bed narrowPeak files with peaks called at an FDR cutoff of 5%:

- K562s: https://www.encodeproject.org/files/ENCFF638QFS/, https://www.encodeproject.org/files/ENCFF784DIM/ \
- HCT116s: https://www.encodeproject.org/files/ENCFF547FKC/, https://www.encodeproject.org/files/ENCFF527GRA/ \
- A549s: https://www.encodeproject.org/files/ENCFF310LGE/, https://www.encodeproject.org/files/ENCFF410KIB/

The two replicates for each cell line were merged using the following commands:

cat replicate1.bed replicate2.bed | sort -k1,1 -k2,2n > combined_peaks.bed \
bedtools merge -i combined_peaks.bed > merged_peaks.bed

In [9]:
#read in merged peaks for each cell line